<a href="https://colab.research.google.com/github/nglglhtr/slack-analysis/blob/master/Kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kernel Slack analysis 

In [287]:
import pandas as pd

### All members of the org (user id, name, email)

In [288]:
users = pd.read_csv('KERNEL 🌱 Member Analytics All time - Sep 10 2020.csv', usecols = ['Name', 'Email','User ID'])
users.set_index('User ID', inplace=True)
users

,Name,Email
User ID,,
U016C8XH6NR,Sachin,sachin@gitcoin.co
U017E2A2KPS,Jo-Ann A. Hamilton,joann.hamilton@gmail.com
U016PNPMFSN,arya,arya@getuni.app
U016C909613,vivek,vivek@gitcoin.co
U016QSTSQQM,Alex Thomas,hello@cawfree.com
...,...,...
U016KTMF17Y,seb,seb@zapper.fi
U016MASHCGN,pakokrew,pakokrew@gmail.com
U01A4R3KSR2,pri,pri@openlaw.io


### Calculating `popularity_index`

- Calculate total messages sent in all channels by each user
- Calculate total reacts received across all the messages sent
- popularity_index = reacts_received / total_messages_sent

In [289]:
import glob
import numpy as np

path = r'channels'
all_files = glob.glob(path + "/*.csv")

most_popular = pd.DataFrame(np.zeros([users.shape[0], 4])*0)
most_popular.columns = ['User ID', 'reacts_received', 'messages_sent', 'popularity_index']
most_popular['User ID'] = users.index
most_popular = most_popular.set_index('User ID')

for channel in all_files:
  df = pd.read_csv(channel, usecols = ['user', 'total_reactions'])
  for index, col in df.iterrows():
    if col['user'] in most_popular.index:
      most_popular.loc[col['user'], 'reacts_received'] += col['total_reactions']
      most_popular.loc[col['user'], 'messages_sent'] = most_popular.loc[col['user'], 'messages_sent'] + 1

most_popular['popularity_index'] = (most_popular['reacts_received'] / most_popular['messages_sent'])
most_popular

,reacts_received,messages_sent,popularity_index
User ID,,,
U016C8XH6NR,170.0,116.0,1.465517
U017E2A2KPS,94.0,55.0,1.709091
U016PNPMFSN,50.0,35.0,1.428571
U016C909613,619.0,138.0,4.485507
U016QSTSQQM,85.0,21.0,4.047619
...,...,...,...
U016KTMF17Y,0.0,0.0,NaN
U016MASHCGN,0.0,3.0,0.000000
U01A4R3KSR2,0.0,0.0,NaN


### Get top 100 most popular (by popularity index)

In [290]:
top_users = most_popular.sort_values('popularity_index', ascending=False).head(100)
top_users

,reacts_received,messages_sent,popularity_index
User ID,,,
U017Q8Y2BPG,31.0,3.0,10.333333
U017X1PE8HX,39.0,4.0,9.750000
U018W6481FC,27.0,3.0,9.000000
U01927CCPFE,15.0,2.0,7.500000
U016FDAC4LT,50.0,7.0,7.142857
...,...,...,...
U017ACMER3P,2.0,3.0,0.666667
U017N4HS1SN,7.0,11.0,0.636364
U01692CEFQF,10.0,16.0,0.625000


In [291]:
fin = pd.merge(top_users, users, on='User ID')
fin

,reacts_received,messages_sent,popularity_index,Name,Email
User ID,,,,,
U017Q8Y2BPG,31.0,3.0,10.333333,lizyang,lizyang@hey.com
U017X1PE8HX,39.0,4.0,9.750000,austin,austin@ethereum.org
U018W6481FC,27.0,3.0,9.000000,tate,tate@arceum.co
U01927CCPFE,15.0,2.0,7.500000,Sam Hatem,sam@outpost-protocol.com
U016FDAC4LT,50.0,7.0,7.142857,shisin21,shisin21@pds.org
...,...,...,...,...,...
U017ACMER3P,2.0,3.0,0.666667,Anne Connelly,anneconnelly@gmail.com
U017N4HS1SN,7.0,11.0,0.636364,Leon Erichsen,leon@radicalxchange.org
U01692CEFQF,10.0,16.0,0.625000,jldcmartins,jldcmartins@gmail.com


In [292]:
fin.to_csv('top_100.csv')